In [1]:
import extract_features as ef

from nltk.tag import hmm
from nltk.probability import LidstoneProbDist

The code in this notebook is based on: http://www.nltk.org/_modules/nltk/tag/hmm.html

In [2]:
def hmm_feat(infilepath):
    """
    tbd
    """

    with open(infilepath, 'r') as f:
        text = f.read()

    tag_set = set()
    lemma_set = set()
    sentences = list()
    
    for line in text.split('\n\n'):
        if line == '-DOCSTART- -X- O O' or '"' in line:
            continue
        else:
            sentence = list()
            sent_items = line.split('\n')
            for item in sent_items:
                if len(item.split(' ')) < 4:
                    continue
                else:
                    token = item.split(' ')[0]
                    ner_tag = ef.clean_ner_tags(item.split(' ')[3])
                    lemma = ef.lemmatizer(token)
                    
                    tag_set.add(ner_tag)
                    lemma_set.add(lemma)
                    
                    lemma_tag = lemma, ner_tag
                    sentence.append(lemma_tag)
                    
            sentences.append(sentence)
        
    return sentences, tag_set, lemma_set

In [3]:
# prep train and test data

train_sents, tags, lemmas = hmm_feat('data/train_reuters.en')

test_sents, test_tags, test_lemmas = hmm_feat('data/test.conll')

# remove empty lists (otherwise: error in testing):

for idx, item in enumerate(train_sents):
    if item == []:
        train_sents.pop(idx)

for idx, item in enumerate(test_sents):
    if item == []:
        test_sents.pop(idx)

In [4]:
# supervised training

trainer = hmm.HiddenMarkovModelTrainer(tags, lemmas)

hmm = trainer.train_supervised(train_sents, estimator=lambda fd, bins: LidstoneProbDist(fd, 0.1, bins),)

In [5]:
# testing

hmm.test(test_sents, verbose=False)

accuracy over 42486 tokens: 93.29
